# Project 03: Gibbs Sampling (Random Algorithm)

In [1]:
import random as random
import numpy as np
import bamnostic as bs
import seqlogo as sl

#import function for building sequence motif & idenfitying seqs matching to motif
from data_readers import get_fasta, get_gff
from seq_ops import get_seq, reverse_complement
from motif_ops import build_pfm, build_pwm, pfm_ic, score_kmer

---
## Implement Gibbs Sampler


Gibbs sampling is a MCMC approach to identify enrichments. Here we will implement a method to identify motifs from a set of regions. 

Important considerations:
- We will need to score each sequence with a PWM using the `score_kmer()` or `score_sequence()` functions
    - You will need to investigate into the help documenation and libraries to identify how best to use these functions. 
- These sites are often not strand-specific and so both scores on the negative as well as positive strand should be considered
- To select a random sequence, use `random.randint()` or `numpy.random.randint()`
- To select a new position $m$ (as defined below) use `random.choices()` or `numpy.random.choice()`  

Assumptions: 
- We know $k$ as the length of expected motif
- Each sequence contains the motif



```
GibbsMotifFinder(DNA, k-length)
    random pick of k-length sequences from each line of DNA as Motifs
    for j ← 1 to 10000 or Motifs stops changing
        i ← Random(N) where N is number of DNA entries
        PWM ← PWM constructed from all Motifs except for Motifi
        Motifi ← select position m from PWM-scored k-mers in DNAi in probabilistic fashion from score distribution
    return PFM
```

Probability of chosing position $m = \frac{A_{m}}{\sum_{l}A_{l}}$ for positions $l$ in DNAi


**Note:** I have also added a function to `motif_ops.py` that will calculate the information content of your motifs. This is useful to observe the progression of your Gibbs sampler as well as a measure of convergence. You can use this function as `IC = pfm_ic(pfm)`. You should expect a slow increase of IC until it plateaus such as in the plot below from your lecture slides:

<center><img src='figures/Gibbs_Sampling.png'/ width=600px></center>

---
# Setting up the data

This BAM file contains a subsampled set of aligned ChIP‑seq reads from a p53 immunoprecipitation experiment in human K562 cells treated with the anthracycline drug daunorubicin. The original SRA experiment SRX5865974 (run SRR9090854) reports 31.2 million Illumina NextSeq 500 ChIP‑seq read pairs from K562 wild‑type cells exposed to daunorubicin, using a p53 antibody to pull down p53‑bound chromatin fragments before library preparation and sequencing. [pmc.ncbi.nlm.nih](https://pmc.ncbi.nlm.nih.gov/articles/PMC4366240/)

### What the data represent

- Biological system: human K562 leukemia cells (Homo sapiens) treated with daunorubicin, a DNA‑damaging chemotherapeutic known to stabilize and activate p53. [pmc.ncbi.nlm.nih](https://pmc.ncbi.nlm.nih.gov/articles/PMC6561911/)
- Assay: ChIP‑seq using an antibody against p53, so reads should be enriched around genomic regions where p53 is bound after drug treatment. [pmc.ncbi.nlm.nih](https://pmc.ncbi.nlm.nih.gov/articles/PMC4526040/)
- Sequencing: Illumina NextSeq 500, with the raw run SRR9090854 corresponding to experiment SRX5865974. [github](https://github.com/ncbi/sra-tools/issues/213)
- BAM file: `SRR9090854.subsampled_5pct.bam` is a coordinate‑sorted alignment file containing ~5% of the original mapped reads, typically created by random down‑sampling the full BAM to reduce file size and speed up exploratory analyses while preserving the overall distribution of p53 binding events. [ecseq](https://www.ecseq.com/support/ngs-snippets/how-to-extract-a-list-of-specific-read-IDs-from-a-BAM-file)

### How `bamnostic` is used and what your code does

The `bamnostic` package provides a pure‑Python interface to BAM files that mirrors the `pysam` API, including an `AlignmentFile` class whose iterator yields `AlignedSegment` objects representing individual aligned reads. In your code: [bamnostic.readthedocs](https://bamnostic.readthedocs.io/en/latest/bamnostic.html)

```python
bam_path = "data/SRR9090854.subsampled_5pct.bam"

seqs = [read.seq for read in bs.AlignmentFile(bam_path)]
```

- `bs.AlignmentFile(bam_path)` opens the BAM file as an `AlignmentFile` object in binary read mode (default `'rb'`), reading the BAM header (reference contigs, read groups, etc.) and preparing a streaming interface to all aligned records. [github](https://github.com/betteridiot/bamnostic/blob/master/docs/source/quickstart.rst)
- Iterating over `AlignmentFile` (`for read in bs.AlignmentFile(...)`) returns each aligned read as a `bamnostic.AlignedSegment` object, which exposes properties analogous to SAM fields such as query name, flags, reference name, position, mapping quality, CIGAR string, and the original sequencing **sequence**. [bamnostic.readthedocs](https://bamnostic.readthedocs.io/en/latest/bamnostic.html)
- The `read.seq` attribute is the query (read) sequence string stored in the BAM, corresponding to the full read sequence (including any unaligned bases), as opposed to `query_alignment_sequence`, which would only contain the aligned portion. [bamnostic.readthedocs](https://bamnostic.readthedocs.io/en/latest/bamnostic.html)
- The list comprehension `[read.seq for read in ...]` consumes the entire BAM stream and collects the nucleotide sequences from every subsampled ChIP‑seq read into a Python list `seqs`, which can then be used for downstream tasks such as motif discovery, k‑mer analysis, or quality checks on read content. [ucdavis-bioinformatics-training.github](https://ucdavis-bioinformatics-training.github.io/2022-Feb-Introduction-To-Python-For-Bioinformatics/python/python5)

In summary, your dataset is a 5% random sample of p53‑ChIP‑seq alignments from daunorubicin‑treated K562 cells, and the `bamnostic` code opens the subsampled BAM and extracts the raw read sequences from each aligned fragment into memory as a list.

In [2]:
bam_path = "data/SRR9090854.subsampled_5pct.bam"

seqs = [read.seq for read in bs.AlignmentFile(bam_path)] 

In [3]:
# Read promoters, store in list of strings
seq_file = "data/GCF_000009045.1_ASM904v1_genomic.fna"
gff_file = "data/GCF_000009045.1_ASM904v1_genomic.gff"

seqss =[]
for name, seq in get_fasta(seq_file):
    for gff_entry in get_gff(gff_file):
        if gff_entry.type == "CDS":
            promoter_seq = get_seq(
                seq,
                gff_entry.start,
                gff_entry.end,
                gff_entry.strand,
                50
            )

            if "AGGAGG" in promoter_seq:
                seqss.append(promoter_seq)

In [4]:
len(seqs), len(seqss)

(3083497, 837)

---
# Project Start

In [5]:
def initialize_random_motif(seqs, k): 
    ''' Function to pick random motifs for each sequence '''

    # Initialize motif list and iterate through each sequence
    motif_sequence_list = []
    for seq in seqs:

        # Pick a random index for start of motif
        motif_start_index = np.random.randint(low = 0, high = len(seq) - k + 1)

        # Append motif to list based on index
        motif_sequence_list.append(seq[motif_start_index:motif_start_index + k])
        
    return motif_sequence_list

In [6]:
def select_new_motif(motif_list, seqs, k):
    '''
    Function iterates 1 time. Picks a sequence to omit, recalculates PWM, calculates kmer score for each possble motif in omitted seq, choses new motif stochastically
    '''

    # Randomly pick a sequence to omit and exclude from calculations
    omitted_seq_ind = np.random.randint(low = 0, high = len(seqs))
    omitted_seq = seqs[omitted_seq_ind]
    temp_motif_list = motif_list[:omitted_seq_ind] + motif_list[omitted_seq_ind+1:]

    # Recalculate PFM with temporary motif list
    new_pfm = build_pfm(temp_motif_list, k)

    # Recalculate PWM with temporary motif list
    new_pwm = build_pwm(new_pfm)

    # Create empty list to hold scores
    scores = []

    # Create sliding window to calculate score of every possible motif for forward (+) and reverse (-) strands
    for i in range(len(omitted_seq) - k + 1):
        motif = omitted_seq[i:i + k]
        r_motif = reverse_complement(motif)

        # Even indexes are (+) strand, odd indexes are (-)
        scores.extend([score_kmer(motif, new_pwm), score_kmer(r_motif, new_pwm)])

    #create sliding window to calculate score of every possible motif for forward strand    
    #reverse_strand = reverse_complement(seqs[omitted_seq_ind])
    #for i in range(len(reverse_strand) - k):
        #r_motif = reverse_strand[i:i + k]
        #scores.append(score_kmer(r_motif, new_pwm))
    
    # Normalize scores with softmax function (2^^score / sum(2^^score))
    #normalize_scores = (2**scores)/sum(2**scores)
    pow_scores = [(2**score) for score in scores]
    total = sum(pow_scores)
    normalize_scores = [s / total for s in pow_scores]

    # Pick a "random" new motif index based on scores
    new_motif_index = np.random.choice(len(normalize_scores), p = normalize_scores)

    # Extract motif from (+) strand if it is an even index (divide by 2 to get actual index from sequence)
    if new_motif_index % 2 == 0:
        new_motif = omitted_seq[int(new_motif_index / 2) : int(new_motif_index / 2) + k]
    
    # Extract motif from (-) strand by reverse complement if it is an odd index (floor division by 2 to get actual index from sequence)
    else:
        new_motif = reverse_complement(omitted_seq[(new_motif_index // 2) : (new_motif_index // 2) + k])

        # Update seqs list with reverse strand
        seqs[omitted_seq_ind] = reverse_complement(omitted_seq)

    #if new_motif_index > (len(normalize_scores)//2):
        #forward_motif_index = new_motif_index - (len(normalize_scores)//2)
        #new_motif = reverse_complement(seqs[omitted_seq_ind][forward_motif_index:forward_motif_index + k])
        #seqs[omitted_seq_ind] = reverse_complement(seqs[omitted_seq_ind])
    #else: 
        #new_motif = seqs[omitted_seq_ind][new_motif_index:new_motif_index + k]

    return omitted_seq_ind, new_motif



In [7]:

def GibbsMotifFinder (seqs, k, seed=None, max_iterations=10000, ic_threshold=1e-8, convergence_iterations=300):

    '''
Function to find a pfm from a list of strings using a Gibbs sampler
    
    Args: 
        seqs (str list): a list of sequences, not necessarily in same lengths
        k (int): the length of motif to find
        seed (int, default=None): seed for np.random
        max_iterations (int, default=10000): number of iterations to stop if there is no convergence
        ic_threshold (float, default=1e-4): IC threshold to consider two values "equal"
        convergence_iterations (int, default=100): number of iterations in a row to consider convergence reached 

    Returns:
        pfm (numpy array): dimensions are 4xlength
    '''
    # Use rng to make random samples/selections/numbers
    # Example: randint = rng.integer(1, 10)
    random.seed(seed) 
    rng = np.random.default_rng(seed)
    
    # Select starting random motifs in each sequence
    motif_list = initialize_random_motif(seqs, k)

    # Create initial PFM
    initial_pfm = build_pfm(motif_list, k)

    # Create PWM from initial motif assignments
    initial_pwm = build_pwm(initial_pfm)

    # Initialize information_content for PFM
    current_ic = pfm_ic(initial_pfm)

    # Initialize iteration and convergence counters
    i = 1
    convergence_counter = 0

    # Iterate until iteration limit
    while i <= max_iterations:

        # Pick a new motif for a randomly selected sequence 
        seq_ind, new_motif = select_new_motif(motif_list, seqs, k)

        # Update motif list with newly picked motif
        motif_list[seq_ind] = new_motif

        # Calculate information content for new PFM
        new_pfm = build_pfm(motif_list, k)
        new_ic = pfm_ic(new_pfm)

        # Calculate difference between old and current ICs to evaluate convergence
        inf_content_delta = abs(new_ic - current_ic)

        # Add +1 to convergence counter if IC difference is smaller than IC threshold
        if inf_content_delta < ic_threshold:
            convergence_counter += 1

        # Restart covergence counter if IC difference is greater than threshold
        else:
            convergence_counter = 0

        # If convergence is reached, return PFM 
        if convergence_counter == convergence_iterations: 
            return new_pfm
        
        # Update current IC and iteration counter for next iteration
        current_ic = new_ic

        if i % 10 == 0:
            print("Iteration:", i)
            print("IC = ", current_ic)
        i += 1
    
    # If loop ends without convergence being reached, output message and return latest PFM
    if i >= max_iterations:
        print("failed to converge...")
        return new_pfm
    

---
# Driver Program
Don't change any of the code here. If you have completed the project by following the coding by contract, the following code should work.

In [8]:
# Run the gibbs sampler:
promoter_pfm = GibbsMotifFinder(seqs,10 )

# Plot the final pfm that is generated: 
sl.seqlogo(sl.CompletePm(pfm = promoter_pfm.T))

Iteration: 10
IC =  0.0836546339553359
Iteration: 20
IC =  0.08365363154744476
Iteration: 30
IC =  0.08365357729891976
Iteration: 40
IC =  0.08365433312564763
Iteration: 50
IC =  0.08365418852608952
Iteration: 60
IC =  0.08365446326377635
Iteration: 70
IC =  0.08365446457325554
Iteration: 80
IC =  0.08365414208565047
Iteration: 90
IC =  0.08365428106867578
Iteration: 100
IC =  0.08365477488865891
Iteration: 110
IC =  0.08365500098243839
Iteration: 120
IC =  0.08365527333917622
Iteration: 130
IC =  0.0836550056149663
Iteration: 140
IC =  0.08365650139590874
Iteration: 150
IC =  0.08365669935745901
Iteration: 160
IC =  0.08365707372862796
Iteration: 170
IC =  0.08365765446894469
Iteration: 180
IC =  0.08365761215575418
Iteration: 190
IC =  0.08365788313480227
Iteration: 200
IC =  0.08365944151007865
Iteration: 210
IC =  0.08365888981669345
Iteration: 220
IC =  0.08365916860395406
Iteration: 230
IC =  0.08366007692812905
Iteration: 240
IC =  0.08366093375096018
Iteration: 250
IC =  0.0836

KeyboardInterrupt: 

In [ ]:
promoter_pfm